# TMDB

In [1]:
import json
with open(r"C:\Users\nbeac\.secret\tmdb_api.json", "r") as f:
    login = json.load(f)
login.keys()

dict_keys(['api-key'])

In [2]:
import tmdbsimple as tmdb
tmdb.API_KEY = login['api-key']

In [3]:
import os, time
import pandas as pd
import datetime as dt
import tzlocal

In [4]:
from tqdm.notebook import tqdm_notebook
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'title_akas_cleaned.csv.gz',
 'title_basics_cleaned.csv.gz',
 'title_ratings_cleaned.csv.gz',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json']

In [5]:
# Load in the dataframe from project part 1 as basics:
basics = pd.read_csv("Data/title_basics_cleaned.csv.gz")
basics.info()
basics.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935 entries, 0 to 2934
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          2935 non-null   object 
 1   titleType       2935 non-null   object 
 2   primaryTitle    2935 non-null   object 
 3   originalTitle   2935 non-null   object 
 4   isAdult         2935 non-null   int64  
 5   startYear       2935 non-null   float64
 6   runtimeMinutes  2935 non-null   int64  
 7   genres          2935 non-null   object 
dtypes: float64(1), int64(2), object(5)
memory usage: 183.6+ KB


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,118,"Comedy,Fantasy,Romance"
1,tt0113026,movie,The Fantasticks,The Fantasticks,0,2000.0,86,"Musical,Romance"
2,tt0113092,movie,For the Cause,For the Cause,0,2000.0,100,"Action,Adventure,Drama"
3,tt0114447,movie,The Silent Force,The Silent Force,0,2001.0,90,Action
4,tt0115937,movie,Consequence,Consequence,0,2000.0,91,Drama


In [6]:
def write_json(new_data, filename, return_data=False): 
    """Appends the input json-compatible data to the json file.
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/
    Args:
        new_data (list or dict): json-compatible dictionary or list
        filename (str): json file to append data to

    Returns:
        return_data(bool): determines if combined data is returned (default =False) 
    """
    import json
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
                file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

    if return_data:
        return file_data

# API Extraction Function

In [7]:
# def get_movie_with_rating(movie_id):
#     #Get the movie object for the current id
#     movie = imdb.Movies(movie_id)
#     # save the .info .releases dictionaries
#     movie_info = movie.info()
#     releases = movie.releases()
#     # Loop through countries in releases
#     for c in releases['countries']:
#         if c['iso_3166_1'] =='US':
#             movie_info['certification'] = c['certification']
#     return movie_info

In [8]:
def get_movie_with_rating(movie_id):
    """Retrieve the dictionary of movie data using tmbdbsimple, including MPAA rating.
    Args:
        movie_id (str): movie-to-retreive's id (from IMDB's tconst column)
        
    Returns:
        dict: dictionary of movie.info() + the certification from movie.releases"""
    # Get the movie object for the current id
    movie = tmdb.Movies(movie_id)
    # save the .info .releases dictionaries
    movie_info = movie.info()
    releases = movie.releases()
    # Loop through countries in releases
    for c in releases['countries']:
        # if the country abbreviation==US
        if c['iso_3166_1' ] =='US':
            ## save a "certification" key in the info dict with the certification
            movie_info['certification'] = c['certification']
    return movie_info

# Before the Loops

In [9]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935 entries, 0 to 2934
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          2935 non-null   object 
 1   titleType       2935 non-null   object 
 2   primaryTitle    2935 non-null   object 
 3   originalTitle   2935 non-null   object 
 4   isAdult         2935 non-null   int64  
 5   startYear       2935 non-null   float64
 6   runtimeMinutes  2935 non-null   int64  
 7   genres          2935 non-null   object 
dtypes: float64(1), int64(2), object(5)
memory usage: 183.6+ KB


In [10]:
YEARS_TO_GET = list(range(2000,2023))
YEARS_TO_GET

[2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 2020,
 2021,
 2022]

# OUTER Loop

## Setting up a Progress Bar

Select a JSON_FILE filename to save the results in progress.
* Check if the file exists.
    * if no:
        * Create the empy JSON_FILE with "with open" that just contains the key "imdb_id"
            
    * if yes:
        * Do nothing!

The code below will identify the folder in the FOLDER we just define above and will name the file based on the current year

In [11]:
# # OUTER LOOP
# for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):

    
#     JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
#     file_exists = os.path.isfile(JSON_FILE)
#     # If it does not exist: create it
#     if file_exists == False:
#     # save an empty dict with just "imdb_id" to the new json file.
#         with open(JSON_FILE,'w') as f:
#             json.dump([{'imdb_id':0}],f)
#     basics = pd.read_csv("Data/title_basics_cleaned.csv.gz")
#     df = basics.loc[basics['startYear']==YEAR].copy()
#     movie_ids = df['tconst'].copy()
    
#     previous_df = pd.read_json(JSON_FILE)
#     movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

# # INNER Loop
#     for movie_id in tqdm_notebook(movie_ids_to_get,
#                                   desc=f'Movies from {YEAR}',
#                                   position=1,
#                                   leave=True):
#         # Attempt to retrieve then data for the movie id
#         try:
#             temp = get_movie_with_rating(movie_id)  #This uses your pre-made function
#             # Append/extend results to existing file using a pre-made function
#             write_json(temp,JSON_FILE)
#             # Short 20 ms sleep to prevent overwhelming server
#             time.sleep(0.02)

#         # If it fails,  make a dict with just the id and None for certification.
#         except Exception as e:
#             continue


    
#     final_year_df = pd.read_json(JSON_FILE)
    
#     csv_fname = f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz"
#     final_year_df.to_csv(csv_fname, compression="gzip", index=False)
    

In [12]:
## Setting booleans to control which method of saving data
ADVANCED_MOVIE_ID_WORKFLOW=False
ERROR_LOG = "tmdb_api_errors.json"
with open(ERROR_LOG,'w') as f:
    json.dump([{'movie id':'','error':''}],f)

In [13]:
# Start of OUTER loop
if ADVANCED_MOVIE_ID_WORKFLOW==True:
    print('- Since ADVANCED_MOVIE_ID_WORKFLOW is True, will save each movie to disk.')
else:
    print('- Since ADVANCED_MOVIE_ID_WORKFLOW is False, will append each movie to list.')

now = dt.datetime.now(tz=tzlocal.get_localzone())
print(f"\n[i] API Calls started on {now.strftime('%m-%D-%Y')} @ {now.strftime('%I:%M:%S %p')}")



for YEAR in tqdm_notebook(YEARS_TO_GET,desc='YEARS',
                          position=0):
    

    #Saving new year as the current df
    df = basics.loc[ basics['startYear']==YEAR].copy()
    # saving movie ids to list
    movie_ids = df['tconst'].copy()#.to_list()
    
    
   ## if saving each movie to disk
    if ADVANCED_MOVIE_ID_WORKFLOW:

        #Defining the JSON file to store results for year
        JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'

        # Check if file exists
        file_exists = os.path.isfile(JSON_FILE)


        # If it does not exist: create it
        if file_exists == False:
        # save an empty dict with just "imdb_id" to the new json file.
            with open(JSON_FILE,'w') as f:
                json.dump([{'imdb_id':0}],f)


        # Load existing data from json into a dataframe called "previous_df"
        previous_df = pd.read_json(JSON_FILE)


        # filter out any ids that are already in the JSON_FILE
        movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

    ## if append to list insteead of saving .json
    else:
        ## Make list to append data to instead of saving to file
        year_data = []
        ## use all movie_ids from basicsß
        movie_ids_to_get = movie_ids
        
        
        
    #Get index and movie id from list
    # INNER Loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        # Attempt to retrieve then data for the movie id
        try:
            temp = get_movie_with_rating(movie_id)  #This uses your pre-ma    de function
            # Append/extend results to existing file using a pre-made function
            
            ## Save to json_file if advanced workflow
            if ADVANCED_MOVIE_ID_WORKFLOW:
                
                write_json(temp,JSON_FILE)
                
            else:
                year_data.append(temp)
                
            
            # Short 10 ms sleep to prevent overwhelming server
            time.sleep(0.01)

        # If it fails,  append error message with id to json
        except Exception as e:
            temp = {'movie_id':movie_id,'error':str(e)}
            write_json(temp, ERROR_LOG)
            continue

    ## FINAL SAVING OF YEAR DATA
    if ADVANCED_MOVIE_ID_WORKFLOW:
        final_year_df = pd.read_json(JSON_FILE)
        
    else:
        final_year_df = pd.DataFrame(year_data)
    
    ## save compressed csv
    csv_fname = f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz"
    final_year_df.to_csv(csv_fname, compression="gzip", index=False)

- Since ADVANCED_MOVIE_ID_WORKFLOW is False, will append each movie to list.

[i] API Calls started on 08-08/19/22-2022 @ 10:14:50 AM


YEARS:   0%|          | 0/23 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/1409 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/1526 [00:00<?, ?it/s]

Movies from 2002: 0it [00:00, ?it/s]

...

Movies from 2018: 0it [00:00, ?it/s]

Movies from 2019: 0it [00:00, ?it/s]

Movies from 2020: 0it [00:00, ?it/s]

Movies from 2021: 0it [00:00, ?it/s]

Movies from 2022: 0it [00:00, ?it/s]

In [14]:
tmdb_results_2000 = pd.read_csv('Data/final_tmdb_data_2000.csv.gz', )
tmdb_results_2000.info()
tmdb_results_2000.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1203 entries, 0 to 1202
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  1203 non-null   bool   
 1   backdrop_path          632 non-null    object 
 2   belongs_to_collection  110 non-null    object 
 3   budget                 1203 non-null   int64  
 4   genres                 1203 non-null   object 
 5   homepage               66 non-null     object 
 6   id                     1203 non-null   int64  
 7   imdb_id                1203 non-null   object 
 8   original_language      1203 non-null   object 
 9   original_title         1203 non-null   object 
 10  overview               1179 non-null   object 
 11  popularity             1203 non-null   float64
 12  poster_path            1063 non-null   object 
 13  production_companies   1203 non-null   object 
 14  production_countries   1203 non-null   object 
 15  rele

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,False,/vMFs7nw6P0bIV1jDsQpxAieAVnH.jpg,NaN,10000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10402, '...",NaN,62127,tt0113026,en,The Fantasticks,...,0,86,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Try to remember the first time magic happened,The Fantasticks,False,5.500,22,NaN
1,False,NaN,NaN,0,"[{'id': 878, 'name': 'Science Fiction'}]",NaN,110977,tt0113092,en,For the Cause,...,0,100,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,The ultimate showdown on a forbidden planet.,For the Cause,False,5.100,8,NaN
2,False,NaN,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",NaN,442869,tt0116391,hi,Gang,...,0,152,"[{'english_name': 'Hindi', 'iso_639_1': 'hi', ...",Released,NaN,Gang,False,0.000,0,NaN
3,False,/n4GJFGzsc7NinI1VeGDXIcQjtU2.jpg,NaN,150000,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",http://www.wkw-inthemoodforlove.com/,843,tt0118694,cn,花樣年華,...,12854953,99,"[{'english_name': 'Cantonese', 'iso_639_1': 'c...",Released,"Feel the heat, keep the feeling burning, let t...",In the Mood for Love,False,8.108,1941,PG
4,False,NaN,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,49511,tt0118852,en,Chinese Coffee,...,0,99,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,There's a fine line between friendship and bet...,Chinese Coffee,False,6.900,46,R


In [15]:
tmdb_results_2001 = pd.read_csv(r"C:\Users\nbeac\OneDrive\Documents\GitHub\Dojo-Project-03\Data\final_tmdb_data_2001.csv.gz")
tmdb_results_2001.info()
tmdb_results_2001.head()

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\nbeac\\OneDrive\\Documents\\GitHub\\Dojo-Project-03\\Data\\final_tmdb_data_2001.csv.gz'